In [1]:
import numpy as np
from PIL import Image
import os
import random
import glob
import keras 
from keras import Model
from keras.layers import Input, Dense, ReLU ,Conv2D, Flatten , Add, Softmax , AveragePooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.losses import binary_crossentropy , categorical_crossentropy

Using TensorFlow backend.


In [2]:
def layer_type1(inputs,filter_size):
    x = Conv2D(kernel_size=(3,3),
               filters= filter_size,
               padding="same",input_shape = (256,256))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def layer_type2(inputs,filter_size=16):
    x = layer_type1(inputs,filter_size=filter_size)
    
    x = Conv2D(kernel_size=(3,3),
               filters= filter_size,
               padding="same")(x)
    
    x = BatchNormalization()(x)
    
    out = Add()([x, inputs])
    
    return out

def layer_type3(inputs,filter_size):
    
    x = layer_type1(inputs,filter_size=filter_size)
    
    x = Conv2D(kernel_size=(3,3),
               filters= filter_size,
               padding="same")(x)
    
    x = BatchNormalization()(x)
    
    x = AveragePooling2D(pool_size=(2,2),strides=2)(x)
    
    y = Conv2D(kernel_size=(1,1),
               filters= filter_size,
               padding="same",strides=2)(inputs)
    y = BatchNormalization()(y)
    
    out = Add()([x, y])
    
    return out

def layer_type4(inputs,filter_size=512):
    x = layer_type1(inputs,filter_size=filter_size)
    x = Conv2D(kernel_size=(16,16),
               filters= filter_size,
               padding="same")(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    return x

In [20]:
def create_SRNet():
    init_input = Input(shape=(256,256,1))
    
    L1 = layer_type1(init_input,filter_size=64)
    L2 = layer_type1(L1,filter_size=16)
    
    L3 = layer_type2(L2)
    L4 = layer_type2(L3)
    L5 = layer_type2(L4)
    L6 = layer_type2(L5)
    L7 = layer_type2(L6)
    
    L8 = layer_type3(L7,filter_size=16)
    L9 = layer_type3(L8,filter_size=64)
    L10 = layer_type3(L9,filter_size=128)
    L11 = layer_type3(L10,filter_size=256)
    
    L12 = layer_type4(L11,filter_size=512)
    
    #L13 = Flatten()(L12)
    L14 = Dense(2)(L12)
    output = Softmax()(L14)
    
    model = Model(init_input,output,)
    model._name = "SRNet"
    model.compile(optimizer = Adam(lr=0.01) , 
               loss = "binary_crossentropy" , metrics = ['accuracy'])
    
    return model

In [21]:
model = create_SRNet()
model.summary()

Model: "SRNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 256, 256, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 256, 256, 64) 256         conv2d_26[0][0]                  
__________________________________________________________________________________________________
re_lu_12 (ReLU)                 (None, 256, 256, 64) 0           batch_normalization_26[0][0]     
______________________________________________________________________________________________

In [ ]:
train = os.listdir('/home/iamsmit96/dataset/boss/train/')
validation = os.listdir('/home/iamsmit96/dataset/boss/validation/')
test = os.listdir('/home/iamsmit96/dataset/boss/test/')
validation

In [ ]:
pwd

In [ ]:
os.listdir('/home/iamsmit96/dataset/boss/test/')

In [ ]:
def get_data(typee,listt):
    image_data =[]
    label=[]
    if listt == train:
        num = 100
    else:
        num = 100
    for i in range(0,num):
        image1 = np.asarray(Image.open('/home/iamsmit96/dataset/boss/{}/cover/{}'.format(typee,listt[i])))  
        image2 = np.asarray(Image.open('/home/iamsmit96/dataset/boss/{}/stego/{}'.format(typee,listt[i])))
        image_data.append([image1,0])
        image_data.append([image2,1])
    return image_data

In [ ]:
path = '/home/iamsmit96/dataset/boss/'

In [7]:
def get_DATA(type_of_data):
    image_data=[]
    path = '/home/iamsmit96/dataset/boss/'
    image_data =[]
    if type_of_data == 'train':
        directory_t_c = glob.glob(path + 'train/cover/*')[:150]
        for i in range(0,len(directory_t_c)):
            print(i)
            image = np.asarray(Image.open(directory_t_c[i]))
            image_data.append([image,0])
        
        directory_t_s = glob.glob(path + 'train/stego/*')[:250]
        for i in range(0,len(directory_t_s)):
            image = np.asarray(Image.open(directory_t_s[i]))
            image_data.append([image,1])
    
    elif type_of_data == 'validation':
        directory_v_c = glob.glob(path + 'validation/cover/*')[:50]
        for i in range(0,len(directory_v_c)):
            image = np.asarray(Image.open(directory_v_c[i]))
            image_data.append([image,0])
        
        directory_v_s = glob.glob(path + 'validation/stego/*')[:50]
        for i in range(0,len(directory_v_s)):
            image = np.asarray(Image.open(directory_v_s[i]))
            image_data.append([image,1])
            
    else:
        directory_te_c = glob.glob(path + 'test/cover/*')[:50]
        for i in range(0,len(directory_te_c)):
            image = np.asarray(Image.open(directory_te_c[i]))
            image_data.append([image,0])
        
        directory_te_s = glob.glob(path + 'test/stego/*')[:50]
        for i in range(0,len(directory_te_s)):
            image = np.asarray(Image.open(directory_te_s[i]))
            image_data.append([image,1])
    random.seed(101)
    return random.sample(image_data,len(image_data))

In [ ]:
%%time
#train_set = get_DATA('train')

In [8]:
%%time
#validation = get_DATA('validation')

CPU times: user 40.5 ms, sys: 36.8 ms, total: 77.3 ms
Wall time: 56.8 s


In [9]:
%%time
#test = get_DATA('test')

CPU times: user 37 ms, sys: 41.7 ms, total: 78.7 ms
Wall time: 1min 11s


In [10]:
x_train = []
y_train = []
x_val = []
y_val = []
x_test =[]
y_test = []

In [11]:
for feature, label in train_set:
        x_train.append(feature)
        y_train.append(label)

In [12]:
for feature, label in validation:
        x_val.append(feature)
        y_val.append(label)

In [13]:
for feature, label in test:
        x_test.append(feature)
        y_test.append(label)

In [18]:
x_train = np.array(x_train)
y_train = np.array(y_train)

np.save("x_train.npy",x_train)
np.save("y_train.npy",y_train)

In [19]:
x_val = np.array(x_val)
y_val = np.array(y_val)

np.save("x_val.npy",x_val)
np.save("y_val.npy",y_val)

x_test = np.array(x_test)
y_test = np.array(y_test)

np.save("x_test.npy",x_train)
np.save("y_test.npy",y_test)

In [22]:
%%time
#model.fit(x_train,y_train,batch_size=20,validation_data=(x_val,y_val),
         epochs=10,use_multiprocessing=True, workers=8)

Epoch 1/10
20/20 [==============================] - 3211s 161s/step - loss: 0.9939 - accuracy: 0.4875 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 2/10
20/20 [==============================] - 3205s 160s/step - loss: 0.7247 - accuracy: 0.4400 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 3/10
20/20 [==============================] - 3211s 161s/step - loss: 0.6974 - accuracy: 0.5025 - val_loss: 7.6685 - val_accuracy: 0.5000
Epoch 4/10
20/20 [==============================] - 3208s 160s/step - loss: 0.6951 - accuracy: 0.5300 - val_loss: 6.3680 - val_accuracy: 0.5400
Epoch 5/10
20/20 [==============================] - 3208s 160s/step - loss: 0.6940 - accuracy: 0.5225 - val_loss: 5.1345 - val_accuracy: 0.5000
Epoch 6/10
20/20 [==============================] - 3205s 160s/step - loss: 0.6948 - accuracy: 0.5275 - val_loss: 1.0041 - val_accuracy: 0.5000
Epoch 7/10
20/20 [==============================] - 3203s 160s/step - loss: 0.6935 - accuracy: 0.4875 - val_loss: 0.7906 - val_accuracy:

<img src="SRNet_Architecture.png" alt="MarineGEO circle logo" style="height: 600px; width:900px;"/>